# Import packages
- 출처
    - 표본코호트DB -> 데모 다운로드 
- 환자 ID: RN_INDI
- 진료명세서 ID: RN_KEY
    - 약물명세서 M60과 연동
    - 요양기관 INST와 연동 (병원코드 통해서)

In [10]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [4]:
# read excel files from the folder 'data'

df_P20 = pd.read_excel('data_demo/약국명세서(P20).xlsx')
df_M60 = pd.read_excel('data_demo/약물명세서(M60).xlsx')
df_INST = pd.read_excel('data_demo/요양기관(INST).xlsx')
df_M20 = pd.read_excel('data_demo/진료명세서(M20).xlsx') # 메인 데이터 (진료명세서)
df_D1 = pd.read_excel('data_demo/인구정보1.xlsx')
df_D2 = pd.read_excel('data_demo/인구정보2.xlsx')

c:\Users\lesga\anaconda3\envs\Insurance\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\lesga\anaconda3\envs\Insurance\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\lesga\anaconda3\envs\Insurance\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\lesga\anaconda3\envs\Insurance\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\lesga\anaconda3\envs\Insurance\lib\site

In [5]:
df_pharmacy = df_P20.copy()
df_drug = df_M60.copy()
df_hospital = df_INST.copy()
df_treat = df_M20.copy() # 메인 데이터 (진료명세서)
df_patient_1 = df_D1.copy()
df_patient_2 = df_D2.copy()

In [6]:
"""
진료명세서(M20) 데이터 필터링

- 세 자리 질병코드만 남김
- 분석대상 질병코드만 남김 
"""

print('전체 질병코드 개수: ', len(df_treat['SICK_SYM1'].unique()))

# filter dataframe by the condition that the column value of 'SICK_SYM1' has the length of 3
df_treat_filtered = df_treat[df_treat['SICK_SYM1'].str.len() == 3]
print('세 자리 질병코드 개수: ', len(df_treat_filtered['SICK_SYM1'].unique()))

# filter df_treat by disease_list
disease_list = ['I20', 'I49', 'I25', 'I50', 'I48', 'I21', 'I51', 'I47', 'I63', 'I67', 'I69', 'I61', 'I65', 'E11', 'E14', 'E10', 'E13', 'I10', 'I11', 'I13', 'I15', 'I12', 'E04', 'E03', 'E05', 'E07', 'E06', 'J45', 'J41', 'J42', 'J15', 'J18', 'J44', 'J46', 'J47', 'J43', 'J12', 'D13', 'C16', 'C34', '143', 'D12', 'C18', 'C22', 'D134', 'D34', 'C73', 'D44']
df_treat_filtered = df_treat_filtered[df_treat_filtered['SICK_SYM1'].isin(disease_list)]
print('분석대상 질병코드 개수: ', len(df_treat_filtered['SICK_SYM1'].unique()))

전체 질병코드 개수:  3437
세 자리 질병코드 개수:  320
분석대상 질병코드 개수:  30


# 진료명세서 데이터 수집 (X변수)

## 질병별 환자 주머니 만들기 

In [7]:
disease_ID = 'SICK_SYM1'
patient_ID = 'RN_INDI'

In [11]:
"""
딕셔너리 만들기 (disease_dict) - key: 질병ID, value: 환자ID 리스트
"""

# create a dict where the key is the "disease_ID" and the value is the "list of patient_ID"
disease_dict = {}
for i in tqdm(range(len(df_treat))):
    if df_treat.loc[i, disease_ID] not in disease_dict:
        disease_dict[df_treat.loc[i, disease_ID]] = [df_treat.loc[i, patient_ID]]
    else:
        disease_dict[df_treat.loc[i, disease_ID]].append(df_treat.loc[i, patient_ID])

"""
딕셔너리 만들기 (disease_df_dict) - key: 질병ID, value: 환자ID 리스트를 담고 있는 데이터프레임(df_treat), grouped by 환자ID, sorted by 날짜, dropped index
"""

# create a dict where the key is the "disease_ID" and the value is the "dataframe filtered" by the value of disease_dict
disease_df_dict = {}
for key in tqdm(disease_dict):
    disease_df_dict[key] = df_treat[df_treat[patient_ID].isin(disease_dict[key])].groupby(patient_ID).apply(lambda x: x.sort_values(by='RN_KEY')).reset_index(drop=True).groupby(patient_ID).apply(lambda x: x).sort_values(by=[patient_ID, 'RN_KEY'])


100%|██████████| 3437/3437 [00:48<00:00, 71.38it/s] 


In [13]:
disease_df_dict['I20']

,RN_INDI,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,MCARE_SUBJ_CD,SICK_SYM1,SICK_SYM2,HSPTZ_PATH_TYPE,OFIJ_TYPE,...,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,FST_HSPTZ_DT,EDC_ADD_RT,SPCF_SYM_TYPE,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,72225,2003100665044,46463,_x0032_0031020,_x0030_3,_x0030_1,E782,NaN,NaN,_x0030_,...,1,0,_x0035_,NaN,0.25,NaN,22520,11260,11260,_x0032_003
1,72225,2003110627319,46463,_x0032_0031114,_x0030_3,_x0030_1,I693,E785,NaN,_x0030_,...,1,0,_x0035_,NaN,0.25,NaN,50830,25410,25420,_x0032_003
2,72225,2004010382244,46463,_x0032_0040128,_x0030_3,_x0030_1,E785,I693,NaN,_x0030_,...,1,0,_x0035_,NaN,0.25,NaN,50810,25400,25410,_x0032_004
3,72225,2004020208187,46463,_x0032_0040227,_x0030_3,_x0030_1,I693,E785,NaN,_x0030_,...,1,0,_x0035_,NaN,0.25,NaN,47360,23680,23680,_x0032_004
4,72225,2004030778515,46463,_x0032_0040330,_x0030_3,_x0030_1,I10,I693,NaN,_x0030_,...,1,0,_x0035_,NaN,0.25,NaN,48080,24040,24040,_x0032_004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,925523,2015112343297,46555,_x0032_0151116,_x0030_3,_x0030_5,S3350,M5312,NaN,_x0030_,...,1,0,_x0031_,NaN,0.15,NaN,15210,4500,10710,_x0032_015
3879,925523,2015120493703,81249,_x0032_0151231,_x0030_3,_x0030_1,J209,J304,NaN,_x0030_,...,1,3,_x0031_,NaN,0.15,NaN,11570,1500,10070,_x0032_015
3880,925523,2015120675379,81249,_x0032_0151229,_x0030_3,_x0031_5,N_,NaN,NaN,_x0030_,...,1,30,_x0031_,NaN,0.15,NaN,10000,1500,8500,_x0032_015
3881,925523,2015121276046,81249,_x0032_0151208,_x0030_3,_x0030_5,M79198,F_,NaN,_x0030_,...,1,3,_x0031_,NaN,0.15,NaN,12830,1500,11330,_x0032_015


## 1. 환자 선별하기
1. 첫 발병 이전 진료명세서 70개 이상
2. 첫 발병 이후 진료기간 2년 이상
    - after_dict에서 첫 발병 이후 2년동안의 명세서만 가져오니까 그걸로 걸러 

### before_dict

In [14]:
"""
딕셔너리 만들기 (before_dict) - key: 질병ID, value: dict{RN_INDI: 첫 발병 이전 명세서 list}
"""

before_dict = {}
for key in tqdm(disease_df_dict): # 질병 for loop
    df_disease = disease_df_dict[key]
    dict_patient = {}
    for patient in df_disease[patient_ID].unique(): # 환자 for loop
        df_patient = df_disease[df_disease[patient_ID] == patient]
        df_patient = df_patient.reset_index(drop=True)
        # find the row index where disease_ID == key for the first time
        idx = df_patient[df_patient[disease_ID] == key].index[0]

        # 첫 발병 이전의 데이터
        df_before = df_patient.iloc[:idx]
        # 70개 필터링
        RN_KEY_list = df_before['RN_KEY'].tolist()
        if len(RN_KEY_list) >= 70:
            dict_patient[patient] = RN_KEY_list

    if dict_patient != {}:
        before_dict[key] = dict_patient
    

100%|██████████| 3437/3437 [00:20<00:00, 168.56it/s]


In [36]:
with open('pickle/before_dict.pkl', 'wb') as f:
    pickle.dump(before_dict, f)

### after_dict

In [37]:
"""
딕셔너리 만들기 (after_dict) - key: 질병ID, value: {RNINDI: 첫 발병 이후 명세서 list}
"""

after_dict = {}
for key in tqdm(before_dict): # 질병 for loop
    filtered_patients = before_dict[key].keys()
    df_disease = disease_df_dict[key]
    dict_patient = {}
    for patient in df_disease[patient_ID].unique(): # 환자 for loop
        # 첫 발병 이전 명세서 70개 이상인 환자들만 필터링
        if patient in filtered_patients:
            df_patient = df_disease[df_disease[patient_ID] == patient]
            df_patient = df_patient.reset_index(drop=True)
            # find the row index where disease_ID == key for the first time
            idx = df_patient[df_patient[disease_ID] == key].index[0]

            # 첫 발병 이후의 데이터
            df_after = df_patient.iloc[idx:] # len==0인 경우 없음
            # 질병A에 해당하는 데이터만 가져오기
            df_after = df_after[df_after[disease_ID] == key]

            # 첫 발병 이후 2년동안의 데이터만 가져오기
            # get the first value of RN_KEY
            df_after['date'] = df_after['RN_KEY'].astype(str).str[:8].astype(int)
            start_date = df_after.iloc[0]['date']
            df_after['start_date'] = start_date
            end_date = start_date + 20000
            # drop rows where RN_KEY exceeds end_date
            df_after = df_after[df_after['date'] < end_date]

            dict_patient[patient] = df_after['RN_KEY'].tolist()


    if dict_patient != {}:
        after_dict[key] = dict_patient


100%|██████████| 2399/2399 [00:28<00:00, 84.33it/s] 


In [39]:
with open('pickle/after_dict.pkl', 'wb') as f:
    pickle.dump(after_dict, f)

In [14]:
# """
# 딕셔너리 만들기 (disease_df_dict_filtered) - key: 질병ID, value: 환자ID 리스트를 담고 있는 데이터프레임(df_treat), grouped by 환자ID, sorted by 날짜, dropped index, filtered by 첫 발병 이전 명세서
# """
# # drop rows after the first occurence of the disease_ID in the grouped dataframe
# # https://stackoverflow.com/questions/70291052/drop-all-rows-after-first-occurrence-of-column-value

# disease_df_dict_filtered = {}
# for key in disease_df_dict:
#     mask = (disease_df_dict[key][disease_ID].eq(key)
#                                                                     .groupby(disease_df_dict[key]['RN_INST'])
#                                                                     .transform(lambda x: x.shift(periods=0, fill_value=False).cummax()))  # periods=0 to include the first occurence
#     disease_df_dict_filtered[key] = disease_df_dict[key][~mask]

# """
# 딕셔너리 만들기 (disease_df_dict_filtered_count) - key: 질병ID, value: 환자 ID마다 첫 발병 이전에 명세서 몇 개인지 count
# """

# # group dataframe and count the number of patient_ID in each group
# disease_df_dict_filtered_count = {}
# for key in disease_df_dict_filtered:
#     disease_df_dict_filtered_count[key] = disease_df_dict_filtered[key].groupby(patient_ID)['RN_KEY'].count()

# """
# 딕셔너리 만들기 (disease_df_dict_filtered_count_filtered) - key: 질병ID, value: 첫 발병 이전에 명세서 70개 이상인 환자ID 리스트
# """

# # filter the pandas series whose value is more than 70 and get the index
# disease_df_dict_filtered_count_filtered = {}
# for key in disease_df_dict_filtered_count:
#     disease_df_dict_filtered_count_filtered[key] = disease_df_dict_filtered_count[key][disease_df_dict_filtered_count[key] >= 70].index


# """
# """
# """
# """


# """
# 딕셔너리 만들기 (disease_df_dict_filtered) - key: 질병ID, value: 전체 데이터 중 앞서 구한 환자ID가 존재하는 데이터만 가져옴
# """

# # create a dict where the key is the "disease_ID" and the value is the "dataframe filtered" by the value of disease_dict 
# disease_df_dict_filtered = {}
# for key in disease_df_dict_filtered_count_filtered:
#     disease_df_dict_filtered[key] = df_treat[df_treat[patient_ID].isin(disease_df_dict_filtered_count_filtered[key])].sort_values(by=[patient_ID, 'RN_KEY'])

# # save disease_df_dict_filtered as a pickle file
# with open('pickle/disease_df_dict_filtered.pkl', 'wb') as f:
#     pickle.dump(disease_df_dict_filtered, f)
    
# """
# 딕셔너리 만들기 (disease_df_dict_filtered_70) - key: 질병ID, value: 전체 데이터 중 앞서 구한 환자ID가 존재하는 데이터만 가져옴 (첫 발병 이후 명세서만 가져옴)
# """

# # drop rows before the first occurence of the disease_ID in the grouped dataframe
# # https://stackoverflow.com/questions/70291052/drop-all-rows-after-first-occurrence-of-column-value

# disease_df_dict_filtered_70 = {}
# for key in disease_df_dict_filtered:
#     mask = (disease_df_dict_filtered[key][disease_ID].eq(key)
#                                                         .groupby(disease_df_dict_filtered[key]['RN_INST'])
#                                                         .transform(lambda x: x.shift(periods=0, fill_value=False).cummax())) # periods=0 to include the first occurence
#     disease_df_dict_filtered_70[key] = disease_df_dict_filtered[key][mask]

# """
# 딕셔너리 만들기 (disease_df_dict_filtered_70_2) - key: 질병ID, value: 전체 데이터 중 앞서 구한 환자ID가 존재하는 데이터만 가져옴 (첫 발병 이후 2년동안의 명세서만 가져옴)
# """

# # filter by RN_KEY (2 years)
# # replace dataframes with the filtered dataframes

# disease_df_dict_filtered_70_2 = {}
# for key in disease_df_dict_filtered_70:
#     df_list = []
#     if len(disease_df_dict_filtered_70[key]) != 0: # if the dataframe is not empty
#         for name, item in disease_df_dict_filtered_70[key].groupby(patient_ID): # 사람마다 for loop
#             item['date'] = item['RN_KEY'].astype(str).str[:8].astype(int)
#             start_date = item.iloc[0]['date']
#             item['start_date'] = start_date
#             end_date = start_date + 20000
#             item_filtered = item[(item['date'] >= start_date) & (item['date'] <= end_date)]
#             df_list.append(item_filtered)
#         filtered_df = pd.concat(df_list)
#         disease_df_dict_filtered_70_2[key] = filtered_df

# # save disease_df_dict_filtered_70_2 as a pickle file
# with open('pickle/disease_df_dict_filtered_70_2.pkl', 'wb') as f:
#     pickle.dump(disease_df_dict_filtered_70_2, f)

# 2. 치료비 데이터 가져오기 (Y변수)

## 진료비 (M) 가져오기 (disease_df_dict_y)

In [16]:
"""
딕셔너리 만들기 (disease_df_dict_y) - key: 질병ID, value: 진료비 항목 3개만 존재하는 데이터
"""

disease_df_dict_y = {}
for key in tqdm(before_dict): # 질병 for loop
    filtered_patients = before_dict[key].keys()
    df_disease = disease_df_dict[key]
    dict_patient = {}
    for patient in df_disease[patient_ID].unique(): # 환자 for loop
        # 첫 발병 이전 명세서 70개 이상인 환자들만 필터링
        if patient in filtered_patients:
            df_patient = df_disease[df_disease[patient_ID] == patient]
            df_patient = df_patient.reset_index(drop=True)
            # find the row index where disease_ID == key for the first time
            idx = df_patient[df_patient[disease_ID] == key].index[0]

            # 첫 발병 이후의 데이터
            df_after = df_patient.iloc[idx:]
            # 질병A에 해당하는 데이터만 가져오기
            df_after = df_after[df_after[disease_ID] == key]

            # 첫 발병 이후 2년동안의 데이터만 가져오기
            # get the first value of RN_KEY
            df_after['date'] = df_after['RN_KEY'].astype(str).str[:8].astype(int)
            start_date = df_after.iloc[0]['date']
            df_after['start_date'] = start_date
            end_date = start_date + 20000
            # drop rows where RN_KEY exceeds end_date
            df_after = df_after[df_after['date'] < end_date]

            dict_patient[patient] = df_after[[patient_ID, 'RN_KEY', disease_ID, 'start_date', 'ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']]


    if dict_patient != {}:
        # combine all the dataframes for each patient into one dataframe
        df = pd.concat(dict_patient.values())
        disease_df_dict_y[key] = df


100%|██████████| 2399/2399 [00:29<00:00, 81.42it/s] 


In [17]:
disease_df_dict_y['B351'] 

,RN_INDI,RN_KEY,SICK_SYM1,start_date,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT
113,30622,2009011646569,B351,20090116,11930,3500,8430
407,46233,2013090634432,B351,20130906,13190,1500,11690
123,79207,2010081360880,B351,20100813,15330,4500,10830
125,79207,2010090673993,B351,20100813,10700,3200,7500
131,79207,2010111927816,B351,20100813,15330,4500,10830
...,...,...,...,...,...,...,...
162,909438,2015121568230,B351,20150302,40950,0,40950
393,925523,2015081567096,B351,20150815,10000,1500,8500
394,925523,2015091975036,B351,20150815,13300,1500,11800
108,951170,2013090496090,B351,20130904,18020,5400,12620


## 약값 (P) 가져오기 (medicine_df_dict_y)

In [20]:
"""
딕셔너리 만들기 (medicine_df_dict_y) - key: 질병ID, value: 약값 항목 3개만 존재하는 데이터, filtered by 2년
"""

# filter columns: ED_RC_TOT_AMT, EDC_SBA, EDC_INSUR_BRDN_AMT (약값 항목 3개만 가져오기)

medicine_df_dict_y = {}
for key in tqdm(disease_df_dict_y):

    # 약값 항목 3개만 있는 데이터
    df = df_pharmacy[df_pharmacy['SICK_SYM1'] == key][[patient_ID, 'RN_KEY', 'SICK_SYM1', 'ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']]

    # attach start_date to the dataframe
    VLOOKUP_df = disease_df_dict_y[key][[patient_ID, 'start_date']].drop_duplicates()
    medicine_df_dict_y[key] = df.merge(VLOOKUP_df, on=[patient_ID], how='left').dropna() # 이 때 start_date에 결측치가 발생하는 애들은 분석 대상이 아니므로 제거 (첫 발병 이전 명세서가 70개 미만인 환자들임)
    
    # filter 2 years
    df_list = []
    if len(medicine_df_dict_y[key]) != 0:
        for name, item in medicine_df_dict_y[key].groupby(patient_ID): # 사람마다 for loop
            item['date'] = item['RN_KEY'].astype(str).str[:8].astype(int)
            start_date = item.iloc[0]['start_date']
            end_date = start_date + 20000
            item_filtered = item[(item['date'] >= start_date) & (item['date'] <= end_date)] # filter by RN_KEY (2 years)
            df_list.append(item_filtered)
        filtered_df = pd.concat(df_list)
        medicine_df_dict_y[key] = filtered_df


100%|██████████| 2399/2399 [00:25<00:00, 93.03it/s] 


In [21]:
medicine_df_dict_y['B351']

,RN_INDI,RN_KEY,SICK_SYM1,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,start_date,date
281,79207,2010092111333,B351,69680,20900,48780,20100813.0,20100921
282,79207,2010111847489,B351,69680,20900,48780,20100813.0,20101118
316,79207,2011011775341,B351,69840,20900,48940,20100813.0,20110117
317,79207,2011030894569,B351,69840,20900,48940,20100813.0,20110308
302,114537,2011071145104,B351,28950,8600,20350,20110703.0,20110711
...,...,...,...,...,...,...,...,...
381,835408,2013080795699,B351,16420,4900,11520,20110917.0,20130807
382,835408,2013080971789,B351,15430,4600,10830,20110917.0,20130809
390,835408,2013090275118,B351,22540,6700,15840,20110917.0,20130902
391,835408,2013091174768,B351,22540,6700,15840,20110917.0,20130911


## 진료비,약값 (y_dict)

In [23]:
"""
딕셔너리 만들기 (y_treat) - key: 질병ID, value: 환자별로 진료비 항목 세 가지 group sum한 데이터프레임
"""

y_treat = {}
for key in tqdm(disease_df_dict_y):
    y_treat[key] = disease_df_dict_y[key].groupby(patient_ID)[['ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']].sum()

"""
딕셔너리 만들기 (y_medicine) - key: 질병ID, value: 환자별로 약값 항목 세 가지 group sum한 데이터프레임
"""

y_medicine = {}
for key in tqdm(medicine_df_dict_y):
    y_medicine[key] = medicine_df_dict_y[key].groupby(patient_ID)[['ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']].sum()

"""
딕셔너리 만들기 (y_dict) - key: 질병ID, value: 환자별로 진료비,약값 모두 합친 데이터
"""

# add two dataframes
y_dict = {}
for key in tqdm(y_treat):
    a = y_treat[key].reset_index()
    b = y_medicine[key].reset_index()
    y_dict[key] = a.add(b, fill_value=0)

100%|██████████| 2399/2399 [00:01<00:00, 1619.26it/s]


In [25]:
y_dict['B351'].head()

,RN_INDI,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT
0,109829.0,290970.0,87100.0,203870.0
1,160770.0,42140.0,10100.0,32040.0
2,239818.0,437040.0,130400.0,306640.0
3,275519.0,31810.0,9500.0,22310.0
4,450592.0,59300.0,17700.0,41600.0


In [49]:
# save y_dict as a pickle file into the folder 'data'
pd.to_pickle(y_dict, 'pickle/y_dict.pkl')

# 질병 A 발병 이전 정보 (before_dict 업데이트)

In [28]:
# read pickle files from the folder 'data'
before_dict = pd.read_pickle('pickle/before_dict.pkl') # key: 질병코드, value: {RNINDI : RN_KEY}
after_dict = pd.read_pickle('pickle/after_dict.pkl')   # key: 질병코드, value: {RNINDI : RN_KEY}
y_dict = pd.read_pickle('pickle/y_dict.pkl')           # key: 질병코드, value: dataframe with cols [RN_KEY, Y1, Y2, Y3]

# to be created
before_dict_new = before_dict.copy()                 # key: 질병코드, value: {RNINDI : [SICK_SYM, 약물코드]}
after_dict_new = after_dict.copy()                   # key: 질병코드, value: {RNINDI : [SICK_SYM, 요양일수, 입내원일부, Demo features]}

In [29]:
"""
가져올 정보 어디에 있는지 확인
"""

# find df from df_list that contains MDCARE_DD_CNT, VSHSP_DD_CNT, FROM_CD, SPCF_SYM_TYPE, MCARE_SUBJ_CD, HSPTZ_PATH_TYPE, OPRTN_YN

features = ['MDCARE_DD_CNT', 'VSHSP_DD_CNT', 'FROM_CD', 'SPCF_SYM_TYPE', 'MCARE_SUBJ_CD', 'HSPTZ_PATH_TYPE', 'OPRTN_YN']
df_list = [df_pharmacy, df_drug, df_hospital, df_treat, df_patient_1, df_patient_2]
df_list_name = ['df_pharmacy', 'df_drug', 'df_hospital', 'df_treat', 'df_patient_1', 'df_patient_2']

for f in features:
    for df, df_name in zip(df_list, df_list_name):
        if f in df.columns:
            print(df_name, 'has', f)
    print('')

print('!! FROM_CD does not exist in any df !!')

df_pharmacy has MDCARE_DD_CNT
df_treat has MDCARE_DD_CNT

df_pharmacy has VSHSP_DD_CNT
df_treat has VSHSP_DD_CNT


df_pharmacy has SPCF_SYM_TYPE
df_treat has SPCF_SYM_TYPE

df_pharmacy has MCARE_SUBJ_CD
df_treat has MCARE_SUBJ_CD

df_pharmacy has HSPTZ_PATH_TYPE
df_treat has HSPTZ_PATH_TYPE

df_pharmacy has OPRTN_YN
df_treat has OPRTN_YN

!! FROM_CD does not exist in any df !!


In [30]:
for CODE in tqdm(before_dict.keys()): # 질병코드
    for RN_INDI in before_dict[CODE].keys(): # 환자번호

        DICT_RNINDI_RNKEY = before_dict[CODE] # 여기서 RN_KEY를 [SICK_SYM, 약물코드]로 바꾸기
        RN_KEY = DICT_RNINDI_RNKEY[RN_INDI]

        # filter df_treat by RN_KEY
        df_treat_filtered = df_treat[df_treat['RN_KEY'].isin(RN_KEY)]
        # get SICK_SYM1 as a list from df_treat_filtered
        SICK_SYM1 = df_treat_filtered['SICK_SYM1'].tolist()

        # filter df_drug by RN_KEY
        df_drug_filtered = df_drug[df_drug['RN_KEY'].isin(RN_KEY)]
        # get GNL_NM_CD as a list from df_drug_filtered
        GNL_NM_CD = df_drug_filtered['GNL_NM_CD'].tolist()

        # update before_dict_new
        before_dict_new[CODE][RN_INDI] = [SICK_SYM1, GNL_NM_CD]

100%|██████████| 2399/2399 [01:48<00:00, 22.16it/s] 


In [ ]:
# save before_dict_new as pickle file in the folder 'data'
pd.to_pickle(before_dict_new, 'pickle/before_dict_new.pkl')

# 질병 A 발병 이후 정보 (after_dict 업데이트)

In [31]:
# loop through CODE, RN_INDI in after_dict
for CODE in tqdm(after_dict.keys()):
    for RN_INDI in after_dict[CODE].keys():

        DICT_RNINDI_RNKEY = after_dict[CODE] # 여기서 RN_KEY를 [SICK_SYM, 약물코드]로 바꾸기
        RN_KEY = DICT_RNINDI_RNKEY[RN_INDI]

        # 1. filter df_treat by RN_KEY
        df_treat_filtered = df_treat[df_treat['RN_KEY'].isin(RN_KEY)]

        # get information as lists from df_treat_filtered
        SICK_SYM1 = df_treat_filtered['SICK_SYM1'].tolist()
        # get most frequent value
        SICK_SYM1 = max(set(SICK_SYM1), key=SICK_SYM1.count)
        MDCARE_DD_CNT, VSHSP_DD_CNT = df_treat_filtered['MDCARE_DD_CNT'].tolist(), df_treat_filtered['VSHSP_DD_CNT'].tolist()
        HSPTZ_PATH_TYPE, OPRTN_YN = df_treat_filtered['HSPTZ_PATH_TYPE'].tolist(), df_treat_filtered['OPRTN_YN'].tolist()

        # 가져 온 정보 변환하기
        # sum lists
        MDCARE_DD_CNT, VSHSP_DD_CNT = sum(MDCARE_DD_CNT), sum(VSHSP_DD_CNT)
        # get the first element 
        HSPTZ_PATH_TYPE = HSPTZ_PATH_TYPE[0]
        # get 1 if any element is 1, else 0
        OPRTN_YN = 1 if 1 in OPRTN_YN else 0

        # 2. filter df_patient_1 by RN_INDI
        df_patient_1_filtered = df_patient_1[df_patient_1['RN_INDI'] == RN_INDI]
        # get information as lists from df_patient_1_filtered
        BTH_YYYY = df_patient_1_filtered['BTH_YYYY'].tolist()
        # get the most frequent value
        BTH_YYYY = max(set(BTH_YYYY), key=BTH_YYYY.count)

        # 3. filter df_patient_2 by RN_INDI
        df_patient_2_filtered = df_patient_2[df_patient_2['RN_INDI'] == RN_INDI]
        # get information as lists from df_patient_2_filtered
        SEX = df_patient_2_filtered['SEX'].tolist()
        # get most frequent values
        SEX = max(set(SEX), key=SEX.count)

        # 4. filter df_hospital by RN_INST and STD_YYYY
        RN_INST = df_treat_filtered['RN_INST'].tolist()
        STD_YYYY = df_treat_filtered['STD_YYYY'].tolist()
        # for loop 
        # each pair of elements from RN_INST and STD_YYYY
        # filter df_hospital by RN_INST and STD_YYYY
        # merge df_hospital_filtered with df_hospital_filtered_merged
        df_hospital_filtered_merged = pd.DataFrame()
        for RN_INST, STD_YYYY in zip(RN_INST, STD_YYYY):
            df_hospital_filtered = df_hospital[(df_hospital['RN_INST'] == RN_INST) & (df_hospital['STD_YYYY'] == STD_YYYY)]
            df_hospital_filtered_merged = pd.concat([df_hospital_filtered_merged, df_hospital_filtered])
        # get information as lists from df_hospital_filtered_merged
        SGG_INST = df_hospital_filtered_merged['SGG_INST'].tolist()
        # get most frequent values
        SGG_INST = max(set(SGG_INST), key=SGG_INST.count)

        # 5. update after_dict_new
        Query_1 = SICK_SYM1
        Query_2 =[MDCARE_DD_CNT, VSHSP_DD_CNT]
        Features = [HSPTZ_PATH_TYPE, OPRTN_YN, SGG_INST]
        Demo_Features = [BTH_YYYY, SEX]
        after_dict_new[CODE][RN_INDI] = [Query_1, Query_2, Features, Demo_Features]

100%|██████████| 2399/2399 [10:33<00:00,  3.79it/s] 


In [ ]:
# save after_dict_new as pickle file in the folder 'data'
pd.to_pickle(after_dict_new, 'pickle/after_dict_new.pkl')

In [52]:
print(before_dict_new['I20'])
print(before_dict_new['L218'])

{435300: [['I500', 'J209', 'J45', 'J45', 'J45', 'I500', 'I500', 'J209', 'J45', 'M518', 'J45', 'J209', 'J45', 'J45', 'J45', 'J45', 'J45', 'J209', 'J45', 'M544', 'I500', 'J45', 'S220', 'J45', 'M544', 'M750', 'M17', 'M17', 'M545', 'M17', 'M170', 'M17', 'M750', 'M459', 'I500', 'I500', 'I500', 'I500', 'I500', 'M199', 'I500', 'I500', 'I500', 'I500', 'M17', 'M17', 'M17', 'M17', 'M179', 'M750', 'M478', 'M179', 'M15', 'M17', 'M179', 'M179', 'M15', 'M478', 'K051', 'K050', 'M17', 'K120', 'I500', 'I500', 'I500', 'M17', 'K120', 'K050', 'M751', 'M17', 'M750', 'M751', 'M751', 'M751', 'I110', 'I500', 'I110', 'M750', 'I110', 'M478', 'M17', 'M478', 'M478', 'M478', 'M751', 'I110', 'M478', 'M478', 'M478', 'M17', 'M750', 'M17', 'M751', 'I500', 'M478', 'M17', 'M17', 'M17', 'M17', 'J45', 'I10', 'I110', 'J45', 'J45', 'I110', 'I10', 'I10', 'I10', 'M17', 'M17', 'M17', 'M17', 'M545', 'I10', 'I110', 'M478', 'M545', 'M17', 'M751', 'M545', 'M478', 'M478', 'I110', 'B370', 'G442', 'M478', 'M545', 'M478', 'M545', 'M47

# 각 질병당 환자 통계량 구하기
- 환자 수, 평균 입내원일, (앞/뒤) 명세서 개수, 총합산 치료비 등

In [53]:
# read pickle files from the folder 'data'
before_dict = pd.read_pickle('pickle/before_dict.pkl')              # key: 질병코드, value: dict {RNINDI : RN_KEY}
after_dict = pd.read_pickle('pickle/after_dict.pkl')                # key: 질병코드, value: dict {RNINDI : RN_KEY}
y_dict = pd.read_pickle('pickle/y_dict.pkl')                        # key: 질병코드, value: dataframe [RN_INDI, Y1, Y2, Y3]

before_dict_new = pd.read_pickle('pickle/before_dict_new.pkl')      # key: 질병코드, value: dict {RNINDI : [SICK_SYM, 약물코드]}
after_dict_new = pd.read_pickle('pickle/after_dict_new.pkl')        # key: 질병코드, value: dict {RNINDI : [SICK_SYM, 요양일수, 입내원일부, Demo features]}

In [56]:
# before data와 after_data 환자수가 잘 맞는지 확인

print(before_dict['L218'].keys())
print(after_dict['L218'].keys())

print(before_dict_new['L218'].keys())
print(after_dict_new['L218'].keys())

dict_keys([27490, 79207, 179785, 238542, 266165, 294572, 331278, 412761, 446652, 589540, 591253, 599419, 610592, 700169, 725112, 921924, 949431])
dict_keys([27490, 79207, 179785, 238542, 266165, 294572, 331278, 412761, 446652, 589540, 591253, 599419, 610592, 700169, 725112, 921924, 949431])
dict_keys([27490, 79207, 179785, 238542, 266165, 294572, 331278, 412761, 446652, 589540, 591253, 599419, 610592, 700169, 725112, 921924, 949431])
dict_keys([27490, 79207, 179785, 238542, 266165, 294572, 331278, 412761, 446652, 589540, 591253, 599419, 610592, 700169, 725112, 921924, 949431])


In [57]:
"""
before_dict_new, after_dict_new 질병ID, 환자ID 같은지 확인
"""

# check if keys are the same in before_dict_new and after_dict_new
for key in tqdm(before_dict_new.keys()):
    if before_dict_new[key].keys() != after_dict_new[key].keys():
        print('key is not the same: ', key)

for key in tqdm(before_dict_new.keys()):

    before_dict_new_disease = before_dict_new[key]
    after_dict_new_disease = after_dict_new[key]
    
    # check if keys are the same (환자ID)

    if before_dict_new_disease.keys() != after_dict_new_disease.keys():
        print('key is not the same: ', key)


100%|██████████| 2399/2399 [00:00<00:00, 1196733.50it/s]


In [16]:
"""
환자 수, 평균 입내원일
"""

stats_dict_1 = {}
for disease in tqdm(after_dict_new.keys()):
    per_disease = after_dict_new[disease]
    days = 0
    for patient in per_disease.keys():
        per_patient = per_disease[patient]
        days += per_patient[1][1]
    
    num_patients = len(per_disease.keys())
    avg_days = days / len(per_disease.keys())

    stats_dict_1[disease] = [num_patients, avg_days]

"""
(앞/뒤) 명세서 개수
"""

stats_dict_2 = {}
for disease in tqdm(before_dict.keys()):

    per_disease = before_dict[disease]
    for patient in per_disease.keys():
        per_patient = per_disease[patient]
        num_before = len(per_patient)
        
    per_disease = after_dict[disease]
    for patient in per_disease.keys():
        per_patient = per_disease[patient]
        num_after = len(per_patient)

    stats_dict_2[disease] = [num_before, num_after]

"""
총합산 치료비
"""

stats_dict_3 = {}
for disease in tqdm(y_dict.keys()):

    per_disease = y_dict[disease]
    sum_of_three = per_disease['ED_RC_TOT_AMT'] + per_disease['EDC_SBA'] + per_disease['EDC_INSUR_BRDN_AMT']
    sum_of_three = sum_of_three.sum()

    stats_dict_3[disease] = sum_of_three

"""
세 가지 정보 합치기
"""
# append values of two dictionaries by key
stats_dict = {}
for key in stats_dict_1.keys():
    stats_dict[key] = stats_dict_1[key] + stats_dict_2[key] + [stats_dict_3[key]]

# convert to dataframe
stats_df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['num_patients', 'avg_days', 'num_before', 'num_after', 'sum_of_three'])
stats_df


100%|██████████| 2399/2399 [00:00<00:00, 3207.33it/s]


,num_patients,avg_days,num_before,num_after,sum_of_three
L218,17,3.117647,92,1,1706420.0
B351,48,3.250000,95,1,10027720.0
N_,45,2.177778,228,1,5235210.0
J304,112,2.258929,186,5,11004940.0
J311,13,3.538462,174,1,1333920.0
...,...,...,...,...,...
M2106,1,2.000000,138,1,553200.0
S2210,1,5.000000,231,1,1831620.0
R1010,1,3.000000,106,1,1246390.0
S92290,1,6.000000,345,1,1186260.0


# 환자를 기준으로 데이터 재구성하기 (-> 환자를 SICK_SYM_1 으로 구분 가능하니깐 안해도 괜찮을듯 ??)

In [ ]:
# before_dict      # key: 질병코드, value: dict {RNINDI : RN_KEY}
# after_dict       # key: 질병코드, value: dict {RNINDI : RN_KEY}

# y_dict           # key: 질병코드, value: dataframe [RN_INDI, Y1, Y2, Y3]

# before_dict_new  # key: 질병코드, value: dict {RNINDI : [SICK_SYM, 약물코드]}
# after_dict_new   # key: 질병코드, value: dict {RNINDI : [SICK_SYM, 요양일수, 입내원일부, Demo features]}

In [17]:
data = {}
for disease in tqdm(before_dict_new.keys()):
    per_disease = before_dict_new[disease]  # {RNINDI : [SICK_SYM, 약물코드]}
    for patient in per_disease.keys():
        per_patient = per_disease[patient]  # [SICK_SYM, 약물코드]
        data[patient] = per_patient

100%|██████████| 2399/2399 [00:00<00:00, 399544.76it/s]


In [18]:
# convert to dataframe
data = pd.DataFrame.from_dict(data, orient='index', columns=['SICK_SYM', '약물코드'])
data

,SICK_SYM,약물코드
27490,"[I10, I110, I110, I119, I10, I10, I10, I10, I1...","[_x0031_07601ATB, _x0031_07601ATB, _x0031_0760..."
79207,"[M6583, M65, M772, J03, M544, M544, M6581, J20...","[_x0031_25302ACH, _x0031_01804ACH, _x0031_4850..."
179785,"[J039, H359, H60, L23, G510, H356, E119, E119,...","[_x0031_01405ATR, _x0032_68000ATB, _x0032_7890..."
238542,"[B_, B_, B_, I20, B_, B_, B_, J304, B_, B_, B_...","[_x0031_72301ATB, _x0031_72301ATB, _x0031_0490..."
266165,"[N_, N_, F_, N_, N_, R100, N_, N_, B353, H353,...","[_x0031_72806ATB, _x0032_26802ATB, _x0031_3330..."
...,...,...
515572,"[N_, S626, S626, B_, B351, B351, H162, B351, H...","[_x0031_43504ATB, _x0032_26802ATB, _x0032_7610..."
739454,"[J209, H650, J019, J06, J36, J060, H600, J209,...","[_x0031_01404ATB, _x0031_08103ACH, _x0032_2680..."
881045,"[S626, S40, S626, J068, L701, J20, L500, J381,...","[_x0033_07900ATB, _x0033_07900ATB, _x0032_3440..."
654182,"[J041, B353, J039, J039, B351, J387, H8130, F_...","[_x0032_68000ATB, _x0032_17001ATB, _x0031_0810..."
